### Test of data instantiation

In [1]:
import torch
import numpy
from torch.utils.data import Dataset, DataLoader 
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import pandas as pd
from PIL import Image
import os


In [2]:

        
# img = torch.ones(6, 3)
# labels = [0, 1, 1, 1, 0, 0]

# dataset = ExpDataSet(img, labels)
# print(dataset[4])



In [3]:
# Local paths

label_path = "dataset/datasets/ringmerkingno/ringcodes.csv"

# Make sure to separate csv file on |
df = pd.read_csv(label_path, sep="|")

df.head(10)

,filename,code,color
0,LBNW(J22M)_20080310234907_79.jpg,J22M,NW
1,LBNW(J03Y)_20080311094828_79.jpg,J03Y,NW
2,LBBW(JR0T)_20080320193604_79.jpg,JR0T,BW
3,LBBW(J1EM)_20080321135128_79.jpg,J1EM,BW
4,LBBW(JZ3K)_20111027125715_79.jpg,JZ3K,BW
5,LBBW(JT0G)_20141104000055_79.jpg,JT0G,BW
6,LBPN(J512)_20090309004214_79.jpg,J512,PN
7,LBBW(J9EE)_20101214162613_79.jpg,J9EE,BW
8,LBBW(J9EE)_20101214162645_79.jpg,J9EE,BW
9,LBBW(J00X)_20090309175308_79.jpg,J00X,BW


In [4]:

dl = df.sort_values("filename", ascending=True).reset_index(drop=True)
dl.head(10)
# exp_labels = dl[["code", "color"]]
# exp_labels.head()


,filename,code,color
0,LABW(WT);LBM_20120228142549_79.jpg,WT,BW
1,LAM;RBWN(J0K8)_20140125233654_1116.jpg,J0K8,WN
2,LAM;RBWN(J0K8)_20190120000322_1116.jpg,J0K8,WN
3,LAM;RBWN(JA0H)_20121029120844_1116.jpg,JA0H,WN
4,LANW(CN);LBM_20120227142458_79.jpg,CN,NW
5,LBBO(232)_20121119155111_79.jpg,232,BO
6,LBBO(924)_20120228083159_79.jpg,924,BO
7,LBBO(CZP)_20121119155815_79.jpg,CZP,BO
8,LBBO(CZP)_20121119155827_79.jpg,CZP,BO
9,LBBW(210E)_20240706180050_385.jpg,210E,BW


In [5]:
class ExpDataSet(Dataset):
    def __init__(self, img_path:str, labels_path:str, transform=None, max_n=None):
        self.img_path = img_path
        self.transform = transform
        self.max_n = max_n

        # Retrieve labels and sort them in alphabetical order to match the images
        df = pd.read_csv(labels_path, sep="|")
        df = df.sort_values("filename", ascending=True).reset_index(drop=True)

        # Ensure the data contains the chosen amount of elements
        if max_n is not None:
            df = df[:self.max_n]
        
        # Retrieve images from folder and match them with labels
        self.img_paths = df["filename"].apply(lambda e: os.path.join(img_path, e)).to_list()

        self.labels = list(df[["code","color"]].itertuples(index=False, name=None)) # Labels should be a list of tuples
        
    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        label = self.labels[idx]
        image = Image.open(self.img_paths[idx]).convert('RGB')

        if self.transform: # Add transformation mask to image
            image = self.transform(image)
        
        return image, label

In [55]:
image_path = "dataset/datasets/ringmerkingno/images"
max_n = 10
transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])
exp_dataset = ExpDataSet(img_path=image_path, labels_path=label_path, transform=transform, max_n=max_n)


In [56]:

exp_loader = DataLoader(dataset=exp_dataset, batch_size=64, shuffle=True)

def tensor_to_numpy(img_tensor):
    img_np = img_tensor.detach().cpu()
    img_np = img_np.permute(0, 2, 3, 1).numpy()
    #img_np= (img_np* 255).clip(0, 255).astype('uint8')
    return img_np

In [44]:
from paddleocr import PaddleOCR as POCR

ocr = POCR(use_doc_orientation_classify=True, use_doc_unwarping=True, use_textline_orientation=True)


Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in C:\Users\legom\.paddlex\official_models.
Creating model: ('UVDoc', None)
Using official model (UVDoc), the model files will be automatically downloaded and saved in C:\Users\legom\.paddlex\official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 472.22it/s]
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Using official model (PP-LCNet_x1_0_textline_ori), the model files will be automatically downloaded and saved in C:\Users\legom\.paddlex\official_models.
Creating model: ('PP-OCRv5_server_det', None)
Using official model (PP-OCRv5_server_det), the model files will be automatically downloaded and saved in C:\Users\legom\.paddlex\official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 1953.87it/s]
Creating model: ('PP-OCRv5_server_rec', None)
Using official model (PP-OCRv5_server_rec), the model files w

In [53]:
results = []
for images, labels in exp_loader:
    images = tensor_to_numpy(images)
    for image in images:
        results.append(ocr.predict(image))

In [27]:
print(len(results))
for result in results:
    print(result)




10
[{'input_path': None, 'page_index': None, 'doc_preprocessor_res': {'input_path': None, 'page_index': None, 'input_img': array([[[0.7529412 , ..., 0.75686276],
        ...,
        [0.7294118 , ..., 0.6901961 ]],

       ...,

       [[0.17254902, ..., 0.22745098],
        ...,
        [0.79607844, ..., 0.7921569 ]]],
      shape=(224, 224, 3), dtype=float32), 'model_settings': {'use_doc_orientation_classify': True, 'use_doc_unwarping': True}, 'angle': 270, 'rot_img': array([[[0.        , ..., 0.        ],
        ...,
        [0.73333335, ..., 0.7372549 ]],

       ...,

       [[0.        , ..., 0.        ],
        ...,
        [0.72156864, ..., 0.6784314 ]]],
      shape=(224, 224, 3), dtype=float32), 'output_img': array([[[0, ..., 0],
        ...,
        [0, ..., 0]],

       ...,

       [[0, ..., 0],
        ...,
        [0, ..., 0]]], shape=(224, 224, 3), dtype=uint8)}, 'dt_polys': [], 'model_settings': {'use_doc_preprocessor': True, 'use_textline_orientation': True}, 'text_

In [54]:
print(results[0])

[{'input_path': None, 'page_index': None, 'doc_preprocessor_res': {'input_path': None, 'page_index': None, 'input_img': array([[[0.54509807, ..., 0.5882353 ],
        ...,
        [0.45490196, ..., 0.4862745 ]],

       ...,

       [[0.28627452, ..., 0.28235295],
        ...,
        [0.7058824 , ..., 0.7372549 ]]],
      shape=(800, 800, 3), dtype=float32), 'model_settings': {'use_doc_orientation_classify': True, 'use_doc_unwarping': True}, 'angle': 270, 'rot_img': array([[[0.        , ..., 0.        ],
        ...,
        [0.54509807, ..., 0.5882353 ]],

       ...,

       [[0.        , ..., 0.        ],
        ...,
        [0.4509804 , ..., 0.48235294]]],
      shape=(800, 800, 3), dtype=float32), 'output_img': array([[[0, ..., 0],
        ...,
        [0, ..., 0]],

       ...,

       [[0, ..., 0],
        ...,
        [0, ..., 0]]], shape=(800, 800, 3), dtype=uint8)}, 'dt_polys': [], 'model_settings': {'use_doc_preprocessor': True, 'use_textline_orientation': True}, 'text_det